In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nx = ny = 128
dx = dy = 1.0
total_step = 1000
dt = 1.0e-2
A = 2.0
mobility = 1.0
grad_coef = 1.0
c0 = 0.5

fig = plt.figure(figsize=(5,5))
cc = np.linspace(0.01, 0.99, 100);
plt.plot(cc, cc**2*(1.-cc)**2 ,color='black')
plt.plot(c0, c0**2*(1.-c0)**2 ,color='r',marker='o',markersize=10)
plt.xlabel('Concentration c')
plt.ylabel('Chemical free energy density')
plt.show()

In [ ]:
con = np.zeros([nx, ny]) # concentration
dfdcon = np.zeros([nx, ny]) # chemical part of diffusion potential

In [ ]:
con = c0 + 0.01 * (0.5 - np.random.rand(nx, ny))
plt.imshow(con, cmap='bwr')
plt.title('initial concentration')
plt.colorbar()
plt.show() 

In [ ]:
def calc_wave_vector(nx, ny, dx, dy):
	nx21 = int(nx/2 + 1)
	ny21 = int(ny/2 + 1)
	nx2 = nx + 2
	ny2 = ny + 2
	dkx = (2.0 * np.pi) / (nx * dx)
	dky = (2.0 * np.pi) / (ny * dy)
	kx = np.zeros([nx, ny])
	ky = np.zeros([nx, ny])
	k2 = np.zeros([nx, ny])
	k4 = np.zeros([nx, ny])

	for i in range(1, nx21):
		for j in range(1, ny):
			fk1 = (i - 1) * dkx
			kx[i, :] = fk1
			kx[nx - i, :] = -fk1
	for i in range(1, nx):
		for j in range(1, ny21):
			fk2 = (j - 1) * dky
			ky[:, j] = fk2
			ky[:, ny - j] = -fk2

	k2[:, :] = kx[:, :]**2 + ky[:, :]**2
	return k2

k2 = calc_wave_vector(nx, ny, dx, dy)
k4 = k2 **2

In [ ]:
for istep in range(total_step+1):
    conk = np.fft.fftn(con) # Fourier transform c_t
    dfdcon = (2.0 * con * (1 - con)**2 -2.0 * con**2 * (1 - con)) # Chemical part of diffusion potential
    dfdconk = np.fft.fftn(dfdcon)  # Fourier transform of the chemical part
    nummer = dt * mobility * A * k2 * dfdconk
    denom = 1.0 + dt * mobility * grad_coef * k4
    conk = (conk - nummer) / denom # Updated concentration c_t+dt in Fourier space
    con = np.real(np.fft.ifftn(conk)) # Inverse Fourier transform of c_t+dt
    
    if istep % 100 == 0:
        print('nstep = ', istep)
        print('Maximum concentration = ', np.max(con))
        print('Minimum concentration = ', np.min(con))
        plt.imshow(con, cmap='bwr')
        plt.title('concentration of B atom')
        plt.colorbar()
        plt.show() 